In [1]:
import pandas as pd

In [2]:
players = pd.read_csv("../data/players.csv")
teams = pd.read_csv("../data/teams.csv")

In [3]:
players.head()

,player,position,team,shirt_number,dob,age,height,strong_foot,date_joined,date_contract,market_value
0,Thibaut Courtois,Goalkeeper,Real Madrid,1,"May 11, 1992",30,2.00m,left,"Aug 9, 2018","Jun 30, 2026",€60.00m
1,Andriy Lunin,Goalkeeper,Real Madrid,13,"Feb 11, 1999",23,1.91m,right,"Jul 1, 2018","Jun 30, 2024",€2.50m
2,Éder Militão,Centre-Back,Real Madrid,3,"Jan 18, 1998",24,1.86m,right,"Jul 1, 2019","Jun 30, 2025",€60.00m
3,David Alaba,Centre-Back,Real Madrid,4,"Jun 24, 1992",30,1.80m,left,"Jul 1, 2021","Jun 30, 2026",€55.00m
4,Antonio Rüdiger,Centre-Back,Real Madrid,22,"Mar 3, 1993",29,1.90m,right,"Jul 1, 2022","Jun 30, 2026",€40.00m


In [4]:
def convert_numeric_abbr(s):
    mapping = {'k': 1000, 'm': 1000000, 'b': 1000000000}
    # Do a check first so it doesn't choke on valid floats     
    if s[-1].isalpha():
        return float(s[:-1]) * mapping[s[-1].lower()]

    return float(s)

In [5]:
# Wrangling data before moving towards analysis
# we wont be able to sum the market value as it is not an int 
players['market_value'] = players["market_value"].apply(lambda x: x.replace("€",""))
players['market_value'] = players["market_value"].apply(lambda x: convert_numeric_abbr(x))

In [6]:
players.head()

,player,position,team,shirt_number,dob,age,height,strong_foot,date_joined,date_contract,market_value
0,Thibaut Courtois,Goalkeeper,Real Madrid,1,"May 11, 1992",30,2.00m,left,"Aug 9, 2018","Jun 30, 2026",60000000.0
1,Andriy Lunin,Goalkeeper,Real Madrid,13,"Feb 11, 1999",23,1.91m,right,"Jul 1, 2018","Jun 30, 2024",2500000.0
2,Éder Militão,Centre-Back,Real Madrid,3,"Jan 18, 1998",24,1.86m,right,"Jul 1, 2019","Jun 30, 2025",60000000.0
3,David Alaba,Centre-Back,Real Madrid,4,"Jun 24, 1992",30,1.80m,left,"Jul 1, 2021","Jun 30, 2026",55000000.0
4,Antonio Rüdiger,Centre-Back,Real Madrid,22,"Mar 3, 1993",29,1.90m,right,"Jul 1, 2022","Jun 30, 2026",40000000.0


### The Final Required Query

In [7]:
final_query = players.groupby(["team","position"]).aggregate({"market_value":"sum"})
final_query = final_query.reset_index()
final_query.to_csv("../data/final_query.csv")

Since we have our required table, let's try to analyse the table a little bit and try to understand which position for which team has the highest market value. This is obviously biased as positions like midfield and forward require more players to play than let's say Goalkeeper. Since there are more players, the total market value would be more for these positions. But it will be interesting to see which team's which position has highest market value.

In [8]:
import altair as alt
final_query1 = final_query[final_query['market_value'] > 100000]
final_query1["x"] = final_query1['team'] + final_query1['position']
click = alt.selection_multi(encodings=['color'])
timeunit='date'
# EXAMPLE
fig = (
        alt.Chart(final_query1)
        .mark_bar(width=10)
        .encode(
            x = alt.X("x", title="Team & Position"),
            y=alt.Y("market_value", title="Market value (in €)"),
            
            color=alt.value('#fbb4ae'),
            tooltip=[alt.Tooltip('x')]
        )
    ).properties(width=600,title={
      "text": ["Which Team's which positon has the highes market value?"], 
      "subtitle": ["Plot of total market value of teams and positions"]
    },).interactive().configure_axis(
    grid=False
).configure_view(
    strokeWidth=0
)
#fig.save("plots/fig2.html")
fig

/Users/aviarora/Desktop/OneDrive - Georgetown University/BCSG test/laliga_data_scrapping/env/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

After looking at this chart, we can say that we are not able to tell much from this as the data is alot for us to comprehend. Thus we need to selectively filter our data for a better visual. Since the objective is to see which team's which position has more market value, we can very easily filter on market value. 

In [13]:
import altair as alt
final_query2 = final_query[final_query['market_value'] > 50000000]
final_query2["x"] = final_query2['team'] +" " + final_query2['position']
click = alt.selection_multi(encodings=['color'])
timeunit='date'
# EXAMPLE
fig = (
        alt.Chart(final_query2)
        .mark_bar(width=20)
        .encode(
            x = alt.X("x", title="Team & Position"),
            y=alt.Y("market_value", title="Market value (in €)"),
            
            color=alt.value('#fbb4ae'),
            tooltip=[alt.Tooltip('market_value')]
        )
    ).properties(width=600,title={
      "text": ["Which Team's which positon has the highes market value?"], 
      "subtitle": ["Plot of total market value of teams and positions"]
    },).interactive().configure_axis(
    grid=False
).configure_view(
    strokeWidth=0
)
fig.save("../plots/highest_market_values.html")
fig

/var/folders/gc/n0p2_5d52qz9v6vm5nmmszx40000gn/T/ipykernel_90293/2942482980.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_query2["x"] = final_query2['team'] +" " + final_query2['position']
/Users/aviarora/Desktop/OneDrive - Georgetown University/BCSG test/laliga_data_scrapping/env/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)